<a href="https://colab.research.google.com/github/NagulapalliSudarshan/MachineLearning/blob/main/Learning/tensorflow/tensorflow5_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt


**The CIFAR-10 dataset**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

In [ ]:
#load and split dataset
(train_imgs,train_labels),(test_imgs,test_labels) = datasets.cifar10.load_data()

# normalize pixel values to be between 0 and 1
train_imgs,test_imgs = train_imgs/255.0,test_imgs/255.0

class_names = ["airplane","automobile","bird","cat","deer","dog","frog",
               "horse","ship","truck"]


In [ ]:
# seeing the image

img_index=7

plt.imshow(train_imgs[img_index],cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[img_index][0]])
plt.show()

In [4]:
model = models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
Total params: 56320 (220.00 KB)
Trainable params: 56320 

In [7]:
# addding the dense layer

model.add(layers.Flatten())
model.add(layers.Dense(64,activation="relu"))
model.add(layers.Dense(10))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [ ]:
# training the model


model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_imgs,train_labels,
                    epochs=4,
                    validation_data=(test_imgs,test_labels))

In [ ]:
# evaluating the model
test_loss,test_accuracy = model.evaluate(test_imgs,test_labels,verbose=2)
print(test_loss,test_accuracy)

# working with small datasets

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator


# creates data generation object that transforms images
datagen=ImageDataGenerator(
    rotation_range=48,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True,
    fill_mode='nearest',
)

# pick an image to transform
test_img=train_imgs[14]
img = image.img_to_array(test_img)
img=img.reshape((1,)+img.shape)

i=0

for batch in datagen.flow(img,save_prefix="test",save_format="jpeg"):
  plt.figure(i)
  plot=plt.imshow(image.img_to_array(batch[0]))
  i+=1

  if i>4:
    break
plt.show()

#pretrained models

In [17]:
# Pretrained Models

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

keras = tf.keras


In [ ]:
import tensorflow_datasets as ds
ds.disable_progress_bar()

(raw_train,raw_validation,raw_test),metadata = ds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
get_label_name = metadata.features['label'].int2str

for image,label in raw_train.take(5):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [21]:
IMG_SIZE=160

def format_example(image,label):
  image=tf.cast(image,tf.float32)
  image=(image/127.5) -1
  image=tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
  return image,label

In [22]:
train=raw_train.map(format_example)
validation = raw_validation.map(format_example)
test=raw_test.map(format_example)

In [ ]:
for image,label in train.take(5):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))